## Load packages and data

In [1]:
# remove hashtag and run code below to install packages

#!pip install -U sentence-transformers
#!pip install --user annoy

In [2]:
import numpy as np
import pandas as pd
import json
import copy
import time
import sqlalchemy as db
from sklearn.model_selection import train_test_split

from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
val_size_abs = 0.15
test_size    = 0.15
shuffle = True

# Note: the first pid_train contains (1-test_size) percent of the data.
# We need to use val_size so that val_size*(1-test_size) = val_size_abs.
val_size = val_size_abs/(1-test_size)
pid_train, pid_test = train_test_split(range(10**6), test_size=test_size,
                                       shuffle=shuffle, random_state=11)
pid_train, pid_val  = train_test_split(pid_train, test_size=val_size,
                                       shuffle=shuffle, random_state=11)

# Generate playlist vectors using sentence transformers and load into annoy index


In [4]:
####use SQL to generate a list of pid's in training set
# pid_train = []

In [10]:

import sqlalchemy as db
engine = db.create_engine("sqlite:///../spotify_sql.db")
conn = engine.connect()

In [27]:

pid = 0
query = f"SELECT * FROM playlists WHERE pid=0"
mydf = pd.read_sql_query(query, conn)
mydf['name'].values[0]

'Throwbacks'

In [9]:
#dimension of vectors the model uses
f = model.get_sentence_embedding_dimension()

#build database we will load vectors in
#angular means we are using cosine similarty metric
t = AnnoyIndex(f , 'angular')

In [28]:
for pid in pid_train:
    #use SQL to pull name of playlist of given pid
    #store in the variable name pid_name

    query = f"SELECT * FROM playlists WHERE pid={pid}"
    pid_name = pd.read_sql_query(query, conn)['name'].values[0]
    
    #construct vector from playlist name
    v = model.encode(pid_name.lower().strip())
    
    #add playlist to annot
    t.add_item(pid, v)

In [29]:
#number of trees for search
t.build(1024)
t.save('playlist_vectors.ann')

True